In [1]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import numpy

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'

sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util
import re


In [25]:
allImpactMuts = analysis_utils.load_in_df_with_progress(filePath = pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf', nLinesFile = 275000)


/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (36,39,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
endometrialIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'all')
colorectalIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'all')
gliomaIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'all')

In [6]:
allImpactMuts['cancerType'] = allImpactMuts['Tumor_Sample_Barcode'].apply(lambda x:
        'Endometrial Cancer' if x in endometrialIds
        else 'Colorectal Cancer' if x in colorectalIds
        else 'Glioma' if x in gliomaIds
        else None
        )


In [7]:
selectMuts = allImpactMuts[allImpactMuts['cancerType'].notnull()]

In [9]:
relatedGenes = {
    'Endometrial Cancer': ["AKT1","APC","ARID1A","BCOR","CCND1","CDKN2A.p16INK4a","CTCF","CTNNB1",         
"ESR1","FBXW7","FGFR2","KRAS","NRAS","PIK3CA",
                           "PIK3R1","PPP2R1A","PTEN","RB1","RRAS2","SOX17","SPOP","TP53"],
    'Colorectal Cancer':["AMER1","APC","ARID1A","ASXL1","ATM","B2M","BRAF","CDKN2A.p16INK4a","CTNNB1","ELF3","EPHA3","ERBB3","FBXW7","JUN","KRAS","NRAS","PIK3CA","PIK3R1","PTEN","RBM10","RNF43","SMAD2","SMAD3","SMAD4","SOX9","TCF7L2","TP53"],
    'Glioma': ["ARID1A","ARID2","ATRX","BCOR","BRAF","CBL","CDKN1B","CDKN2A.p14arf","CDKN2A.p16INK4a","CDKN2C","CIC","DNMT3A","EGFR","FUBP1","H3F3A","IDH1","KRAS","NF1","NOTCH1","PDGFRA","PIK3CA","PIK3R1","PPM1D","PTEN","PTPN11","RB1","SETD2","STAG2","TP53"]
}

hyperGenes = {
    'Endometrial Cancer': ["ARID5B","B2M","CASP8","MAP3K1","MSH2","MSH6","NF1","RASA1"],
    'Colorectal Cancer': ["CASP8","HLA-A","HLA-B","MSH2"],
    'Glioma': ["MSH6"]
}

weaklyRelatedGenes = {
    'Endometrial Cancer':[ "ARID1B","ATM","ATR","CDKN1B","CDKN2A.p14arf","EPHA3","EPHA5","ERRFI1","INPPL1","JAK1","KMT2B","KMT2D","NFE2L2","ROS1","STAT3","ZFHX3"],
    'Colorectal Cancer':["ARID2","BMPR1A","CDKN1B","ERBB2","GNAS","MAP2K1","MAP2K4","PBRM1","PTCH1","RASA1","TGFBR2"],
    'Glioma':["FGFR1","HLA-B","PBRM1","SMARCB1","TSC1"]
}

In [15]:
tmbInfo = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/tmbInfo.tsv')
tmbDict = dict(zip(tmbInfo['Tumor_Sample_Barcode'], tmbInfo['Nmut_Mb']))

In [ ]:
listOfDicts = []
cntr = 0
for case in set(selectMuts['Tumor_Sample_Barcode']):
    nmut = selectMuts.shape[0]
    caseOncogenicMuts = selectMuts[(selectMuts['Tumor_Sample_Barcode'] == case) & (selectMuts['oncogenic'].notnull())]
    
    #GET TMB
    tmb = None
    if case in tmbDict:
        tmb = tmbDict[case]
    
    #get cancer type
    cancerType = None
    if case in endometrialIds: cancerType = 'Endometrial Cancer'
    if case in colorectalIds: cancerType = 'Colorectal Cancer'
    if case in gliomaIds: cancerType = 'Glioma'
     
    related = relatedGenes[cancerType]
    hyper = hyperGenes[cancerType]
    weak = weaklyRelatedGenes[cancerType]
    
    nRelatedMuts = caseOncogenicMuts[caseOncogenicMuts['Hugo_Symbol'].isin(related)].shape[0]
    nHyperMuts = caseOncogenicMuts[caseOncogenicMuts['Hugo_Symbol'].isin(hyper)].shape[0]
    nWeakMuts = caseOncogenicMuts[caseOncogenicMuts['Hugo_Symbol'].isin(weak)].shape[0]
    
    listOfDicts.append({'cancerType': cancerType, 'nRelated': nRelatedMuts, 'nHyperMuts': nHyperMuts,
    'nWeakMuts': nWeakMuts, 'nOncogenic':caseOncogenicMuts.shape[0], 'tmb': tmb})
        
    cntr += 1
    if cntr%100 == 0: print cntr
df = pd.DataFrame(listOfDicts)

In [24]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/geneMutTypeSummary.tsv', index=False, sep='\t')